**CS4210 Fall 2023 Project Assignment 3 | By: Alexander J Sanna**



**Description:** This is my submission for the CS4210 Assignment three competition. This is a working convolution neural network designed to predict emotions based on 48x48 pixel image input. The competition is designed to have a dataset input of 2304 points representing the image and cooresponding labels (0,1,2) to represent emotions respectively. PYTORCH is the only library used.

The code is littered with comments explaining choices and techniques used.
With any questions please contact ajsanna@cpp.edu

In [1]:
# Storing my data sets in google drive for ease of use.
# To run this code adequately, create a folder: 'datasets' inside of the google colab folder and store data files inside.
# If not running on colab, please import datasets and labels accordingly. This program is desined for file x containing data and file y containing labels.
from google.colab import drive
drive.mount('/content/gdrive')

!ls -l ./gdrive/MyDrive/Colab\ Notebooks/datasets

Mounted at /content/gdrive
total 164269
-rw------- 1 root root     23875 Oct 17 04:57 diabetes2.csv
-rw------- 1 root root     30623 Nov 30 17:11 submission.csv
-rw------- 1 root root  33075636 Nov 18 08:26 test_data.csv
-rw------- 1 root root 135047556 Nov 18 08:26 train_data.csv
-rw------- 1 root root     32350 Nov 20 18:41 train_target.csv


**Step 1: Loading Data into file via PANDAS**

In [2]:
import pandas as pd

# load data from file located on google drive using pandas readcsv method.
train_data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/train_data.csv', header = None).to_numpy()
train_target = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/train_target.csv', header = None).to_numpy().flatten()

val_data = train_data[0:3235]
val_target = train_target[0:3235]
train_data = train_data[3235:]
train_target = train_target[3235:]

#The lines above split the data into validation and training sets to get a sense of how accurate our model is later on when we begin training.

In [3]:
#Verifying Data size coorolations.
print("Train Data: ",train_data.shape)
print("Train Target: ",train_target.shape)
print("Validation Data: ",val_data.shape)
print("Validation Target: ",val_target.shape)


Train Data:  (12940, 2304)
Train Target:  (12940,)
Validation Data:  (3235, 2304)
Validation Target:  (3235,)


**Step 2: Data Preprocessing:** Creating the data loader objects.

In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as data_utils
from torch.nn.functional import normalize

#Device selects GPU for accelerated runtimes.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#this line below converts the numpy array into a tensor datatype. VERY IMPORTANT TRAIN DATA STEP.
train_tensor = torch.from_numpy(train_data).type(torch.float32).view(-1,2304)
target_tensor =  torch.from_numpy(train_target).type(dtype = torch.long)

val_tensor = torch.from_numpy(val_data).type(torch.float32).view(-1,2304)
valid_target =  torch.from_numpy(val_target).type(dtype = torch.long)

#Below is an optional normalization tactic that can be used. I tested and didnt see a large difference.
#mean, std, var = torch.mean(train_tensor), torch.std(train_tensor), torch.var(train_tensor)
#train_tensor = (train_tensor-mean)/std
#train_tensor = normalize(train_tensor, p=1.0, dim = 0)

#This line shapes our data in the 48x48 format for image recognition purposes.
train_tensor = train_tensor.reshape(-1, 1, 48, 48)
val_tensor = val_tensor.reshape(-1, 1, 48, 48)

''' The Following code crates the pytorch dataloader objects for training purposes later.'''
trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_tensor, target_tensor), shuffle=True, batch_size=512)
validloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(val_tensor, valid_target), shuffle=False, batch_size=512)


#Testing the data loader output for both train and validation:
i1, l1 = next(iter(trainloader))
print("Train Labels:", l1.shape)
print("Train Images:", i1.shape)

v1, v2 = next(iter(validloader))
print("Valid Labels:", v2.shape)
print("Valid Images:", v1.shape)



Train Labels: torch.Size([512])
Train Images: torch.Size([512, 1, 48, 48])
Valid Labels: torch.Size([512])
Valid Images: torch.Size([512, 1, 48, 48])


**Code is Organized.**
Lets start building our model.

In [6]:

# Final number of outputs from our CNN.
n_out = 3


# I have experimented with a bunch of different models as this cell will show.
# The one not commented out was the model I was able to reach a maximum accuracy score with.
'''
model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 4, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(144, 72),
            nn.Tanh(),
            nn.Linear(72, 36),
            nn.Tanh(),
            nn.Linear(36, n_out)
)
'''


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 96, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(96, 48, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(48, 32, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.act4 = nn.Tanh()
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = nn.Conv2d(16, 4, kernel_size=3, padding=1)
        self.act5 = nn.Tanh()
        self.pool5 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4, 72)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(72, n_out)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = self.pool4(self.act4(self.conv4(out)))
        out = self.pool5(self.act5(self.conv5(out)))

        #print(out.shape)
        out = out.view(-1, 4) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

model = Net()



'''
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>

        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        #print(out.shape)
        return out + x

import torch.nn.functional as F

class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(1, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))

        self.fc1 = nn.Linear(12 * 12 * n_chans1, 32)
        self.fc2 = nn.Linear(32, n_out)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        #print(out.shape)
        out = out.view(-1, 12 * 12 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        #print(out.shape)
        return out

model = NetResDeep(n_chans1=48, n_blocks=10)

'''


#Move model to GPU for accelerated runtime.
model.to(device)



Net(
  (conv1): Conv2d(1, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): Tanh()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): Tanh()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(48, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): Tanh()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): Tanh()
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): Tanh()
  (pool5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4, out_features=72, bias=True)
  (fc2): Linear(in_features=72, out_fe

**Training Loop**
This is where we will run our train data through the model we have built in predetermined batch sizes. Run our results through a loss function and optimize with standard gradient descent.

In [8]:


learning_rate = .05

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 100



for epoch in range(n_epochs):
    model.train(True)
    for imgs, labels in trainloader:
        imgs, labels = imgs.to(device), labels.to(device)   # move tensors to GPU for computation acceleration

        outputs = model(imgs)
        #print(outputs.shape) #Debugging lol
        train_loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in validloader:
            imgs, labels = imgs.to(device), labels.to(device)   # move tensors to GPU for computation acceleration

            outputs = model(imgs)
            val_loss = loss_fn(outputs, labels)

            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

    print("Epoch: %d, train_loss: %f, val_loss: %f, val_accuracy: %f" % (epoch, float(train_loss), float(val_loss), (correct / total)))

Epoch: 0, train_loss: 1.044097, val_loss: 1.047668, val_accuracy: 0.436167
Epoch: 1, train_loss: 1.056074, val_loss: 1.047877, val_accuracy: 0.436167
Epoch: 2, train_loss: 1.066047, val_loss: 1.047880, val_accuracy: 0.436167
Epoch: 3, train_loss: 1.078505, val_loss: 1.048643, val_accuracy: 0.436167
Epoch: 4, train_loss: 1.042514, val_loss: 1.046804, val_accuracy: 0.436167
Epoch: 5, train_loss: 1.077752, val_loss: 1.048081, val_accuracy: 0.436167
Epoch: 6, train_loss: 1.050908, val_loss: 1.046052, val_accuracy: 0.436167
Epoch: 7, train_loss: 1.078371, val_loss: 1.046007, val_accuracy: 0.436167
Epoch: 8, train_loss: 1.087743, val_loss: 1.048241, val_accuracy: 0.436167
Epoch: 9, train_loss: 1.046343, val_loss: 1.041819, val_accuracy: 0.436167
Epoch: 10, train_loss: 1.071865, val_loss: 1.041049, val_accuracy: 0.436167
Epoch: 11, train_loss: 1.079712, val_loss: 1.035480, val_accuracy: 0.436167
Epoch: 12, train_loss: 1.030807, val_loss: 1.020874, val_accuracy: 0.438022
Epoch: 13, train_loss:

In [ ]:
# The following code is designed to import the test file and run it through our model to log predictions.
# The predictions are then stored in an dictionary and exported to a CSV file for submission.

test_data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/test_data.csv', header = None).to_numpy()
test_tensor = torch.from_numpy(test_data).type(torch.float32).view(-1,2304)
test_tensor = test_tensor.reshape(-1, 1, 48, 48)
test_tensor = test_tensor.to(device)
print(test_tensor.shape)


predictions = model(test_tensor)
_, predicted = torch.max(predictions, dim=1)
print(len(predicted))



torch.Size([3965, 1, 48, 48])
3965


In [ ]:
fields = ['Id', "Category"]
import csv

data = []
val = len(predicted)
#print(val)
for x in range(val):
  data.append( [x, predicted[x].item()])
  #print(x)




with open('/content/gdrive/MyDrive/Colab Notebooks/datasets/submission.csv', 'w') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(fields)
  csvwriter.writerows(data)




In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/submission.csv')
test.head


<bound method NDFrame.head of         Id  Category
0        0         1
1        1         0
2        2         1
3        3         1
4        4         1
...    ...       ...
3960  3960         0
3961  3961         0
3962  3962         1
3963  3963         2
3964  3964         2

[3965 rows x 2 columns]>

In [ ]:
filename = '/content/gdrive/MyDrive/Colab Notebooks/datasets/submission.csv'
# opening the file with w+ mode truncates the file
f = open(filename, "w+")
f.close()